<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/Logistic_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#I'm scared to open it back up

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

In [0]:
#Assignment - metadata from music tracks extracted from wav
#need to clean - might need to directly edit csv
#predict genre from metadata
#what are the best predictors for genre?
#what isn't useful for predicting genre?
#what surprised me about predictions?

In [1]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-02-28 20:57:19--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  26.1MB/s    in 31s     

2019-02-28 20:57:51 (11.0 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [2]:
#The work from last doc in one cell
#Process below

#pass tracks twice to use row 0 as col names
tracks = pd.read_csv('fma_metadata/tracks.csv')
tracks = pd.read_csv('fma_metadata/tracks.csv', names=tracks.iloc[0])

#first three rows are garbage
tracks = tracks.drop([0,1,2]).reset_index()

#appears to be most descriptive set - selected in doc notebook
#our order: album, artist, set, track
columns = [
    'title.1',
    'genres_all',
    'genre_top',
    'duration',
    'listens.1',
    'title',
    'listens',
    'tags',
    'name',
    'location',
]

df = tracks[columns]

df = df.rename(columns = {
    'title.1' : 'track_title',
    'listens.1':'track_listens',
    'title':'album',
    'listens':'album_listens',
    'name': 'artist'})

#clean nans as category 'None' (other than genre_top)
df['album'] = df.album.replace({np.nan:'None'})
df['location'] = df.location.replace({np.nan:'None'})
df['track_title'] = df.track_title.replace({'None':'Untitled'})

#fix dtypes
df['duration'] = df['duration'].astype(int)
df['track_listens'] = df['track_listens'].astype(int)
df['album_listens'] = df['album_listens'].astype(int)

#drop the remaining nans from genre_top
df = df.dropna().reset_index()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:709: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [3]:
df.head()

,index,track_title,genres_all,genre_top,duration,track_listens,album,album_listens,tags,artist,location
0,0,Food,[21],Hip-Hop,168,1293,AWOL - A Way Of Life,6073,[],AWOL,New Jersey
1,1,Electric Ave,[21],Hip-Hop,237,514,AWOL - A Way Of Life,6073,[],AWOL,New Jersey
2,2,This World,[21],Hip-Hop,206,1151,AWOL - A Way Of Life,6073,[],AWOL,New Jersey
3,3,Freeway,[10],Pop,161,50135,Constant Hitmaker,47632,[],Kurt Vile,None
4,9,Street Music,[21],Hip-Hop,207,943,AWOL - A Way Of Life,6073,[],AWOL,New Jersey


In [0]:
df = df.drop(columns=['index'])

In [0]:
#loop wont run for overly large reassignments
df = df.drop(columns=['track_title', 'tags', 'genres_all'])

In [6]:
df.dtypes

genre_top        object
duration          int64
track_listens     int64
album            object
album_listens     int64
artist           object
location         object
dtype: object

In [7]:
objects = df.select_dtypes(include=object)
notobjects = df.select_dtypes(exclude=object)

for label, col in objects.iteritems():
  n = 0
  for types in col.unique():
      col = col.replace({types:n})
      n = n+1
  objects[label] = col

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
#please...
df = pd.concat([objects, notobjects], axis=1)
df.head() #yes we can continue to the regression

,genre_top,album,artist,location,duration,track_listens,album_listens
0,0,0,0,0,168,1293,6073
1,0,0,0,0,237,514,6073
2,0,0,0,0,206,1151,6073
3,1,1,1,1,161,50135,47632
4,0,0,0,0,207,943,6073


In [0]:
df_small = df.sample(n=20000)

In [0]:
#define X and y
X = df_small.drop(columns = ['genre_top'])
y = df_small['genre_top']

In [12]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=15000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

0.3407

In [14]:
X = df_small.drop(columns = ['genre_top', 'album_listens'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=15000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)
#accuracy raised after removing albumlistens

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.3531

In [15]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'track_listens'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=15000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)
#lowered after removing track_listens

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.3207

In [16]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'duration'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=20000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)
#lowered after removing duration

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.3258

In [17]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'location'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=20000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

#location isn't horribly relevant

0.3538

In [18]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'location', 'artist'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=20000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

#slightly lowered after removing artist

0.3514

In [19]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'location', 'album'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs',
                           multi_class = 'multinomial', max_iter=20000)

model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

#lowers accuracy if dropped

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.3461

In [20]:
X = df_small.drop(columns = ['genre_top', 'album_listens', 'location'])
X.head()

,album,artist,duration,track_listens
43388,6275,8119,232,1363
38352,5667,6536,141,379
25385,4032,2369,146,609
13871,2356,2925,341,125
43747,6321,8180,400,314


The most accurate logistic regression model I could make had the features **album name, artist name, track length and number of listens on the sharing service.** Out of a much larger dataset I analyzed these features along with number of listens to the album and the geographical origin of the music. This decision was based around the features selected in the official documentation notebook.

The most relevant features for predicting a song's genre appear to be track length and number of listens, which tank the accuracy to 0.32 if removed (the other drops or raises kept in a 0.345 - 0.055 range ). I hypothesized track length would be important (an experimental song is going to be longer than a pop song) but thought number of listens would just be trivia. Now that I think about it, though, it makes sense that different genres on the service would have significantly different sizes of interest in them.

While track listens are strongly relevant to the model, album listens are strongly irrelevant and misleading to use in predictions. Removing album listens improved my model by 0.01. The only other feature I looked at that increased the accuracy when removed (the geographical origin of the music) did so to a very minor extent. I hypothesize the pool of people who listen to shuffled tracks and mixes from their favorite genre is seperate, and larger, than the pool of people listening to specific albums.